# Introduction to Natural Language Processing 2 Lab03

## Introduction

On this part, we will evaluate a simple semantic search with and without nearest neighbours approximation. We fill first create a searchable index, then evaluate queries on it in terms of Mean Average Precision and speed.

### Create a searchable index

In [1]:
# Use the Beir library to extract the test set of the DBPedia entity dataset.

from beir import util
from beir.datasets.data_loader import GenericDataLoader

def extract_test(dataset):
    """
    Extract the test test set with the Beir library.

    Parameters
    ----------
    dataset : str
        The name of the dataset.

    Returns
    -------
    corpus, queries, qrels : dict
        The test set.
    """
    url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
    data_path = util.download_and_unzip(url, "datasets")
    corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")
    return corpus, queries, qrels

dataset = "dbpedia-entity"
corpus, queries, qrels = extract_test(dataset)

/home/unikarah/.local/lib/python3.10/site-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/4635922 [00:00<?, ?it/s]

What are the three values returned by Beir, and how are they presented ?

...

*Let's see what the first values of each value look like*

In [2]:
list(queries.items())[:10]

[('INEX_LD-2009022', 'Szechwan dish food cuisine'),
 ('INEX_LD-2009039', 'roman architecture'),
 ('INEX_LD-2009053', 'finland car industry manufacturer saab sisu'),
 ('INEX_LD-2009061', 'france second world war normandy'),
 ('INEX_LD-2009062', 'social network group selection'),
 ('INEX_LD-2009063', 'D-Day normandy invasion'),
 ('INEX_LD-2009074', 'web ranking scoring algorithm'),
 ('INEX_LD-2009115', 'virtual museums'),
 ('INEX_LD-2010004', 'Indian food'),
 ('INEX_LD-2010014', 'composer museum')]

In [5]:
list(corpus.items())[:5]

[('<dbpedia:Animalia_(book)>',
  {'text': "Animalia is an illustrated children's book by Graeme Base. It was originally published in 1986, followed by a tenth anniversary edition in 1996, and a 25th anniversary edition in 2012. Over three million copies have been sold.   A special numbered and signed anniversary edition was also published in 1996, with an embossed gold jacket.",
   'title': 'Animalia (book)'}),
 ('<dbpedia:Academy_Award_for_Best_Production_Design>',
  {'text': "The Academy Awards are the oldest awards ceremony for achievements in motion pictures. The Academy Award for Best Production Design recognizes achievement in art direction on a film. The category's original name was Best Art Direction, but was changed to its current name in 2012 for the 85th Academy Awards.  This change resulted from the Art Director's branch of the Academy being renamed the Designer's branch.",
   'title': 'Academy Award for Best Production Design'}),
 ('<dbpedia:An_American_in_Paris>',
  {'tex

In [7]:
qrels['INEX_LD-2009022']

{'<dbpedia:Afghan_cuisine>': 0,
 '<dbpedia:Akan_cuisine>': 0,
 '<dbpedia:Ambuyat>': 0,
 '<dbpedia:American_Chinese_cuisine>': 1,
 '<dbpedia:Ants_climbing_a_tree>': 2,
 '<dbpedia:Baingan_bharta>': 1,
 '<dbpedia:Bamischijf>': 0,
 '<dbpedia:Black_cardamom>': 0,
 '<dbpedia:Brazilian_cuisine>': 0,
 '<dbpedia:British_cuisine>': 0,
 '<dbpedia:Caribbean_cuisine>': 0,
 '<dbpedia:Cellophane_noodles>': 1,
 '<dbpedia:Ceviche>': 0,
 '<dbpedia:Chana_masala>': 0,
 '<dbpedia:Chen_Kenichi>': 1,
 '<dbpedia:Chen_Kenmin>': 1,
 '<dbpedia:Chicago-style_pizza>': 0,
 '<dbpedia:Chicken_(food)>': 0,
 '<dbpedia:Chifle>': 0,
 '<dbpedia:Chili_oil>': 2,
 '<dbpedia:Chinatown,_Los_Angeles>': 0,
 '<dbpedia:Chinatown>': 1,
 '<dbpedia:Chinese_cuisine>': 2,
 '<dbpedia:Churumuri_(food)>': 0,
 '<dbpedia:Cookbook>': 0,
 '<dbpedia:Cooking>': 0,
 '<dbpedia:Couscous>': 0,
 '<dbpedia:Cuban_cuisine>': 0,
 '<dbpedia:Cuisine>': 0,
 '<dbpedia:Cuisine_of_Jharkhand>': 0,
 '<dbpedia:Cuisine_of_the_Southern_United_States>': 0,
 '<dbped

To ease the problem, extract all the document from the corpus which are relevant to at least one query. Then, add 100K random documents which are not relevant to any query. Make sure the process is reproducible by setting the random seed on whatever random sampling method you use.

In [15]:
import random

corpus_reduced_dict = {}


qrels_irrelevant = []
# first we get all the corpus that are relevant to queries
for id, query in queries.items():
    # Get al the relevant content with the id of the query
    qrels_for_id = qrels[id]

    # Get all the queries that are relevant (value 1 or 2)
    qrels_relevant = []
    for id, relevancy in qrels_for_id.items():
        if relevancy > 0:
            qrels_relevant.append(id)
    
    # get the corpus elements corresponding and add them the the reduced corpus
    for id in qrels_relevant:
        corpus_reduced_dict[id] = corpus[id]

corpus_irrelevant = {}
for id, corp in corpus.items():
    if id not in corpus_reduced_dict:
        corpus_irrelevant[id] = corp

corpus_reduced = list(corpus_reduced_dict.values())

# Choose the 100k random documents
random.seed(42)
corpus_random_keys = random.sample(list(corpus_irrelevant), 100000)

for id in corpus_random_keys:
    corpus_reduced.append(corpus[id])
print("Total length of the documents chosen:", len(corpus_reduced))


Total length of the documents chosen: 114877


Now, we should be ready to start experimenting with our smaller dataset. Use the
sentence-transformers library to index your dataset. As queries and documents
are different, use an asymetric similarity models. Pick one model across the
ones proposed. Make sure to document your choice, and why you picked it (because
of accuracy, speed, ...).

**Response:**
On the documentation for asymetric similarity models, we can choose from
multiple models. Ones being for cosine-similarity and the others being tuned for
dot-product. The two models in each categorie having the best accuracies are:
- msmarco-distilbert-base-v4 (for cosine-similarity).
- msmarco-distilbert-base-tas-b (for dot-product).

Here we have short passages, and no very long paragraph. As recommended in the
docummentation we are going to use the cosine-similarity model.

In [11]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('msmarco-distilbert-base-v4')

Embed the reduced corpus and the queries using the chosen model.

In [16]:
corpus_embedings = model.encode(corpus_reduced) # FIXME : run meeeeeeee

KeyboardInterrupt: 

In [14]:

queries_embedings = model.encode(list(queries.values()))

[[ 0.12601008  0.4351175  -0.14884456 ...  0.12693802  0.21826428
  -0.12447575]
 [ 1.3706691   0.35628912 -0.0987019  ...  0.01148714 -0.03544339
   0.69143116]
 [-0.04835236  0.26421046 -0.40233096 ... -0.57687294 -0.16397607
  -0.5554676 ]
 ...
 [-0.36190474 -0.28840816  0.41469216 ... -0.46373257  0.4913169
   0.55152303]
 [ 0.637174    0.41584215 -0.74660397 ...  0.6245678   1.1596165
  -0.18587023]
 [-0.02936348 -0.07553382  0.3569843  ...  0.553844    0.2052955
  -0.7003371 ]]


Using the annotated set of queries, compute the Mean Average Precision (MAP) @100 as well as the average time per query.

In [ ]:
# First let's build the confusion matrix with the result and qrels
# FIXME

# Approximate nearest neighbours

Find a good set of parameters for the chosen ANN library and compute the Mean Average Precision @100.

We chose to use the Faiss library for the next part.

In [ ]:
%pip install faiss

In [ ]:
import faiss

# FIXME ALL OF IT
xb = corpus
xq = queries
queries_list = list(queries.values())
d = # dimension

index = faiss.IndexFlatL2(d)   # build the index
index.add(corpus_reduced)                  # add vectors to the index

k = 4                          # we want to see 4 nearest neighbors
D, I = index.search(corpus_reduced[:5], k) # sanity check
print(I)
print(D)
D, I = index.search(queries_list, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries

explain what the parameters you picked are, and why you chose them